In [0]:
def buscar_deputados():

    import requests
    import json
    from datetime import datetime

    pasta_destino = "/Volumes/workspace/deputados/deputados/bronze/"

    url_api = "https://dadosabertos.camara.leg.br/api/v2/deputados"

    try:
        print("Buscando dados dos deputados...")

        response = requests.get(url_api)
        response.raise_for_status()
        dados_deputados = response.json()

        print(f"Encontrados {len(dados_deputados['dados'])} deputados")

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        nome_arquivo = f"deputados_{timestamp}.json"
        caminho_completo = f"{pasta_destino}/{nome_arquivo}"

        dbutils.fs.mkdirs(pasta_destino)

        with open(caminho_completo, 'w', encoding='utf-8') as f:
            json.dump(dados_deputados, f, ensure_ascii=False, indent=2)

        print(f"✅ Arquivo salvo com sucesso: {caminho_completo}")
        return caminho_completo

    except Exception as e:
        print(f"❌ Erro: {e}")
        return None

In [0]:
caminho_arquivo = buscar_deputados()

In [0]:
# Caminho do seu arquivo JSON
caminho_json = "dbfs:/Volumes/workspace/deputados/deputados/deputados_20250929_204600.json"

# Lê o arquivo JSON
df = spark.read.option("multiline", "true").json(caminho_json)
df.printSchema()


In [0]:
from pyspark.sql import functions as F

# Extrai apenas os IDs do array 'dados'
df_ids = df.select(
    F.explode("dados").alias("deputado")
).select(
    F.col("deputado.id").alias("id")
)

df_ids.show(10)